# Unit 9: LightFM

You almost made it - this is the final lesson and it is also going to be the easiest one.

As you might assume there are a lot of recommender packages in Python out there. And yes, there are. In this lesson we will look at LightFM - an easy to user and lightweight implementation of different algorithms for factorization machines to implement CF, CBF and hybrid recommenders.

Within a few lines of code we set-up, train and use a recommender for recommendations.

* [LightFM on GitHub](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/home.html)

In [1]:
import matplotlib.pyplot as plt

In [2]:
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

/anaconda3/envs/recsys_training/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


## Load Data

In [3]:
data = fetch_movielens(min_rating=4.0)

In [4]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 49906 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 5469 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

## Collaborative Filtering

In [5]:
params = {
    'no_components': 16,
    'loss': 'bpr',
    'learning_rate': 0.05,
    'random_state': 42 
}

epochs = 10

N = 10

In [6]:
cf_model = LightFM(**params)

In [7]:
data['train']

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>

In [8]:
cf_model.fit(data['train'], epochs=epochs, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [9]:
# if we provide training data with evaluation, known positives will be removed
prec_at_N = precision_at_k(cf_model, data['test'], data['train'], k=N)

In [10]:
prec_at_N.mean()

0.11563171

## Hybrid (CF + CBF)

In [11]:
hybrid_model = LightFM(**params)

In [12]:
hybrid_model.fit(data['train'], item_features=data['item_features'],
                 epochs=epochs, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [13]:
prec_at_N = precision_at_k(hybrid_model, data['test'], data['train'], k=N,
                           item_features=data['item_features'])

In [14]:
prec_at_N.mean()

0.11563171

In [15]:
hybrid_model.user_embeddings

array([[-0.09265602, -0.08516123, -0.04532801, ..., -0.18063717,
        -0.369014  , -0.34703383],
       [-0.44146848,  0.5685266 , -0.1575276 , ..., -0.65432096,
         0.04467577, -0.04715269],
       [-0.17290047,  0.19480753, -0.07752099, ..., -0.2385519 ,
         0.04283759, -0.06491522],
       ...,
       [-0.20854989,  0.19796765,  0.12975833, ..., -0.20461978,
        -0.16927364, -0.17153145],
       [ 0.28548828,  0.09094203, -0.13034889, ...,  0.2714032 ,
         0.32988533,  0.463295  ],
       [-0.24552047, -0.1440835 ,  0.03641402, ...,  0.52104855,
        -0.34692684, -0.2511615 ]], dtype=float32)

In [16]:
hybrid_model.item_embeddings

array([[-0.66507524,  0.5648045 ,  0.19520864, ..., -0.23561181,
        -0.91469485, -0.49202445],
       [ 0.07023591, -0.00680864,  0.3785799 , ...,  0.39570287,
        -0.3637755 , -0.18149117],
       [-0.43165162,  0.17838618,  0.09997975, ..., -0.08547973,
        -0.22984311, -0.28508952],
       ...,
       [ 0.01892807,  0.0201646 ,  0.01235028, ...,  0.00639794,
         0.00937711,  0.00821096],
       [-0.02309976, -0.01656645,  0.00743713, ...,  0.00811526,
        -0.02904057, -0.02638749],
       [-0.02616825,  0.01739005, -0.00935894, ...,  0.02403321,
         0.02048195,  0.02280193]], dtype=float32)

In [17]:
hybrid_model.user_biases

array([-3.5688767 , -1.4331167 , -0.92312694, -0.82701963, -2.0466132 ,
       -2.7463157 , -4.2477145 , -1.5374001 , -0.859053  , -3.393117  ,
       -2.6407309 , -1.6692423 , -4.696706  , -2.3476136 , -1.6147959 ,
       -2.924297  , -0.73329043, -3.239044  , -0.6080359 , -1.0972704 ,
       -1.8226485 , -2.188729  , -2.5332165 , -1.9068483 , -2.1414032 ,
       -1.1331643 , -0.51610595, -1.8247143 , -1.1486626 , -1.27917   ,
       -1.1425216 , -0.86499405, -0.720063  , -0.66214067, -0.5053325 ,
       -0.6566818 , -1.354422  , -2.2102244 , -0.6783178 , -0.6021603 ,
       -1.4216981 , -2.6653624 , -3.259275  , -2.5916908 , -1.1831447 ,
       -0.8979107 , -0.6995344 , -1.4936945 , -2.0316238 , -0.71863884,
       -0.57945544, -1.749157  , -0.96160585, -1.4311885 , -0.5870579 ,
       -2.6106236 , -1.7477973 , -2.847791  , -4.4814506 , -3.4268248 ,
       -0.3644964 , -2.9734435 , -1.3161491 , -2.9326506 , -1.9951631 ,
       -0.9629884 , -0.80545425, -0.8292232 , -1.5535241 , -2.08